## Pseudobulk

In [ ]:
adata = sc.read_h5ad('/home/glennrdx/Documents/Research_Project/processed_h5ad/crypt_enriched.h5ad')

In [ ]:
adata

AnnData object with n_obs × n_vars = 27419 × 17573
    obs: 'Sample', 'Diet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'doublet_score', 'predicted_doublet', 'leiden', '_scvi_batch', '_scvi_labels', 'published_annotations', 'EEC_refined_1', 'EEC_refined_final', 'final_annotations', 'major_cell_types'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'Diet_colors', 'Sample_colors', 'final_annotations', 'final_annotations_colors', 'major_cell_types_colors', 'published_annotations_colors', 'umap'
    

In [ ]:
import scipy
# Combine all gene counts of all cells for each sample
# Group by 'Sample' and sum the counts
pseudobulk = adata.to_df().groupby(adata.obs['Sample']).sum()

# Create a new AnnData object for the pseudobulk data with sparse matrix
pseudobulk_adata = sc.AnnData(X=scipy.sparse.csr_matrix(pseudobulk.values))

# Add the sample and diet information to the new AnnData object
pseudobulk_adata.var = adata.var
columns_to_remove = ['mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells']  # Replace with actual column names
# Remove specified columns
pseudobulk_adata.var = pseudobulk_adata.var.drop(columns=columns_to_remove)

pseudobulk_adata.obs['n_genes'] = np.sum(pseudobulk_adata.X > 0, axis=1).A1

pseudobulk_adata.obs['Sample'] = pseudobulk.index
pseudobulk_adata.obs['Diet'] = pseudobulk_adata.obs['Sample'].apply(lambda x: x[:-2])

# Label each sample's cell type as 'Intestinal epithelium'
pseudobulk_adata.obs['major_cell_types'] = 'Intestinal epithelium'

In [ ]:
pseudobulk_adata.write('crypt_enriched_pseudobulk.h5ad')